This notebook is for projecting Project Sidewalk labels to the cleaned pedestrian network.

In [15]:
import geopandas as gpd
from shapely.geometry import Point


In [16]:
#read in ped network
pedestrian_network = gpd.read_file('../data/cleaned-pednetwork/cleaned-pednetwork-full/cleaned-pednetwork-full.shp')

In [17]:
#single out the fclass = crossing
crossings = pedestrian_network[pedestrian_network['fclass'] == 'crossing']

In [20]:
# read in the filtered ramps data
ramps = gpd.read_file('../data/project-sidewalk/project-sidewalk-labels/filtered-ramps/filtered-ramps.shp')

In [21]:
ramps

,id,labelType,streetId,osmWayId,neighborhd,avgImgDate,avgLblDate,severity,temporary,nAgree,nDisagree,nNotsure,clusterSze,userIds,geometry
0,104337968,CurbRamp,22785,424636851,Montlake,2021-06-30 17:00:00.0,2024-05-08 21:57:04.142,2.0,F,1,0,0,1,[8d5ecc00-2b9a-4edf-af7f-ce0ba7edb4a8],POINT (1277926.743 238356.633)
1,104249562,NoCurbRamp,18691,242504567,East Queen Anne,2019-04-30 17:00:00.0,2020-12-20 20:02:50.521,3.0,F,2,0,0,1,[7e87b104-1b6d-4b3b-bde0-1733f7463dab],POINT (1266846.900 234326.846)
2,104354371,CurbRamp,22318,389798783,Interbay,2018-11-29 16:00:00.0,2020-03-13 06:32:54.663,1.0,F,2,0,0,2,"[91376904-0b1e-4a89-a83b-fa5d9c52f20f,12d3f3e7...",POINT (1259453.075 240344.310)
3,104354280,CurbRamp,7343,6465628,Interbay,2019-04-10 17:00:00.0,2020-11-22 05:24:45.404,1.0,F,2,0,0,3,"[12d3f3e7-1635-41ee-84e5-28f4130828d5,f98c9d5b...",POINT (1259460.057 240410.974)
4,104247881,CurbRamp,3950,6406572,East Queen Anne,2019-04-30 17:00:00.0,2021-01-31 10:40:23.757667,1.0,F,4,0,0,3,"[0e1ae564-6d72-4670-98e4-71369cc5ab26,12d3f3e7...",POINT (1265401.015 235613.512)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16699,104253799,CurbRamp,21993,370823239,Lower Queen Anne,2019-04-30 17:00:00.0,2021-04-25 13:33:50.267333,1.0,F,3,0,0,2,"[cc0ae2d2-82fa-4fe9-b80d-322e13e877da,09f7d7cc...",POINT (1265440.008 231494.576)
16700,104254020,CurbRamp,19879,263689012,Lower Queen Anne,2019-04-30 17:00:00.0,2021-04-20 21:24:15.285,1.0,F,1,1,0,1,[09f7d7cc-8c3b-4fa4-9c21-d7808bae8fb2],POINT (1265121.518 231090.310)
16701,104253816,CurbRamp,25112,460421474,Lower Queen Anne,2019-04-30 17:00:00.0,2021-03-17 20:44:07.029,1.0,F,1,0,0,1,[09f7d7cc-8c3b-4fa4-9c21-d7808bae8fb2],POINT (1265063.800 231502.016)
16702,104253741,CurbRamp,19878,263689011,Lower Queen Anne,2018-05-31 17:00:00.0,2019-05-06 20:41:14.433,1.0,F,0,0,0,1,[7b1596af-14d4-4f2b-9e4e-71b1884db836],POINT (1264769.016 230679.764)


In [25]:
def project_points_to_nearest_line_spatial_join(points_gdf, lines_gdf):
    # Spatial join to find the nearest line for each point
    points_gdf['nearest_line_id'] = points_gdf.geometry.apply(
        lambda geom: lines_gdf.distance(geom).idxmin()
    )
    # Merge the points with their corresponding nearest lines
    points_with_lines = points_gdf.merge(
        lines_gdf, left_on='nearest_line_id', right_index=True, suffixes=('', '_line')
    )
    # Project the points onto the nearest line
    points_with_lines['projected_point'] = points_with_lines.apply(
        lambda row: row.geometry_line.interpolate(row.geometry_line.project(row.geometry)),
        axis=1
    )
    # Create a new GeoDataFrame with the projected points
    projected_gdf = gpd.GeoDataFrame(points_with_lines, geometry='projected_point', crs=points_gdf.crs)
    projected_gdf = projected_gdf.drop(columns=['geometry', 'nearest_line_id', 'geometry_line'])
    return projected_gdf


In [29]:
projected_ramps = project_points_to_nearest_line_spatial_join(ramps, crossings)

In [31]:
projected_ramps

,id,labelType,streetId,osmWayId,neighborhd,avgImgDate,avgLblDate,severity,temporary,nAgree,nDisagree,nNotsure,clusterSze,userIds,pednet_id,osm_id,type,fclass,crossing_t,projected_point
0,104337968,CurbRamp,22785,424636851,Montlake,2021-06-30 17:00:00.0,2024-05-08 21:57:04.142,2.0,F,1,0,0,1,[8d5ecc00-2b9a-4edf-af7f-ce0ba7edb4a8],0,1000034200,ped,crossing,m,POINT (1277944.809 238365.143)
8323,104337855,CurbRamp,22785,424636851,Montlake,2019-07-31 17:00:00.0,2020-11-25 15:24:30.154,1.0,F,1,0,0,1,[f98c9d5b-98ff-4965-a49c-037deee24520],0,1000034200,ped,crossing,m,POINT (1277952.509 238346.534)
8362,104337855,CurbRamp,22785,424636851,Montlake,2019-07-31 17:00:00.0,2020-11-25 15:24:30.154,1.0,F,1,0,0,1,[f98c9d5b-98ff-4965-a49c-037deee24520],0,1000034200,ped,crossing,m,POINT (1277952.509 238346.534)
1,104249562,NoCurbRamp,18691,242504567,East Queen Anne,2019-04-30 17:00:00.0,2020-12-20 20:02:50.521,3.0,F,2,0,0,1,[7e87b104-1b6d-4b3b-bde0-1733f7463dab],5,1005641195,ped,crossing,m,POINT (1266851.150 234325.270)
2,104354371,CurbRamp,22318,389798783,Interbay,2018-11-29 16:00:00.0,2020-03-13 06:32:54.663,1.0,F,2,0,0,2,"[91376904-0b1e-4a89-a83b-fa5d9c52f20f,12d3f3e7...",6,1005679677,ped,crossing,m,POINT (1259457.322 240344.547)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16669,104254098,CurbRamp,1664,6362630,Lower Queen Anne,2018-05-31 17:00:00.0,2019-05-06 19:54:50.206,1.0,F,1,0,0,1,[7b1596af-14d4-4f2b-9e4e-71b1884db836],30369,991783917,ped,crossing,u,POINT (1263884.493 233371.405)
16672,104247777,CurbRamp,2675,6383248,East Queen Anne,2019-04-30 17:00:00.0,2020-10-24 12:50:33.454,1.0,F,5,0,0,2,"[0e1ae564-6d72-4670-98e4-71369cc5ab26,12d3f3e7...",30387,991820339,ped,crossing,m,POINT (1266663.248 236473.053)
16685,104266407,NoCurbRamp,21528,351740959,Madrona,2019-05-31 17:00:00.0,2021-03-12 14:04:02.145,2.0,F,1,1,0,1,[6c0f27ba-5dd1-4b59-af68-a7f4533ab0ae],30423,993540552,ped,crossing,u,POINT (1281048.245 226548.081)
16693,104306269,CurbRamp,25137,460423536,Madison Park,2019-07-31 17:00:00.0,2020-05-31 16:16:37.244,1.0,F,1,0,0,1,[cbdba4bd-c537-46c1-b08d-fff21c52fa3f],30448,995449531,ped,crossing,u,POINT (1284485.447 235224.148)


In [32]:
#save projected points
projected_ramps.to_file('../data/project-sidewalk/project-sidewalk-labels/projected/projected-ramps/projected-ramps.shp')

/var/folders/zm/x5dzdlrs5mlbxrm7z5g6p05r0000gn/T/ipykernel_38879/1517409665.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  projected_ramps.to_file('../data/project-sidewalk/project-sidewalk-labels/projected/projected-ramps/projected-ramps.shp')
